## It has code for 2 factors
#### 1. Spell Checking - More spelling error means mostly writeen by un-professionals
####    and may be a fake news
#### 2. Geography

In [877]:
import numpy as np
import pandas as pd

In [878]:
columns=['id','label','statement','subject','speaker','speaker_job','state',
        'party_affiliation','barely_true_count','false_Count',
        'half_true_count','mostly_true_count','pants_on_fire_count','venue_speach'];
df_lair=pd.read_csv('train.tsv',sep='\t',header=None,names=columns,index_col=False);
df_lair.head()

,id,label,statement,subject,speaker,speaker_job,state,party_affiliation,barely_true_count,false_Count,half_true_count,mostly_true_count,pants_on_fire_count,venue_speach
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [879]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Text Proprocessing

#### Function for text preprocessing
###### 1.lowercase the text
###### 2.word tokenization
###### 3.remove stop words and non alphanumeric charaters
###### 4.stemming

In [880]:
from nltk.stem import PorterStemmer,WordNetLemmatizer
def text_preprocessing(df_base):
    df=df_base.copy()
    # lowercase the text
    df['statement']=df['statement'].str.lower()
    # word tokenization
    df['statement']=df['statement'].map(lambda x: nltk.word_tokenize(x))
    # remove stop words and non alphanumeric charaters
    df['statement']=df['statement'].map(lambda x: [w for w in x if (not w in stop_words) and w.isalpha()])
    # lemmatization
    #wordnet_lemmatizer = WordNetLemmatizer()
    #df_train['statement']=df_train['statement'].map(lambda x: wordnet_lemmatizer.lemmatize(x))    
    # stemming
    porter = PorterStemmer()
    df['statement']=df['statement'].map(lambda x: [porter.stem(w) for w in x] )
    return df

#### Calling text_preprocessing function

In [881]:
df_train=text_preprocessing(df_lair)
df_train.head()

,id,label,statement,subject,speaker,speaker_job,state,party_affiliation,barely_true_count,false_Count,half_true_count,mostly_true_count,pants_on_fire_count,venue_speach
0,2635.json,false,"[say, anni, list, polit, group, support, abort...",abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,"[declin, coal, start, start, natur, ga, took, ...","energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"[hillari, clinton, agre, john, mccain, vote, g...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,"[health, care, reform, legisl, like, mandat, f...",health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,"[econom, turnaround, start, end, term]","economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


## Spelling Error Check Factor

###### function spell_check_article - It takes an article as input and returns number of words with spelling error in an article and % of words with errors

In [882]:
from nltk.corpus import wordnet
import enchant
enchant = enchant.Dict("en_US")
def spell_check_article(article):
    article=article.replace('-',' ');
    article_tokenized=nltk.word_tokenize(article)
    total_word_count=len(article_tokenized)
    words_with_error_count=0
    for w in article_tokenized:
        if (not wordnet.synsets(w)) and (not enchant.check(w)):
            words_with_error_count+=1
    return words_with_error_count,(words_with_error_count/total_word_count*100)

In [883]:
spell_check_article('kitten drining choclate malk')

(3, 75.0)

In [884]:
spell_check_article('kitten drining choclate milk')

(2, 50.0)

In [885]:
df_lair['spell_error']=df_lair['statement'].map(lambda x: spell_check_article(x)[0])
df_lair[['statement','spell_error']].head()

,statement,spell_error
0,Says the Annies List political group supports ...,1
1,When did the decline of coal start? It started...,3
2,"Hillary Clinton agrees with John McCain ""by vo...",2
3,Health care reform legislation is likely to ma...,0
4,The economic turnaround started at the end of ...,0


## Word2Vec to create a vector representation of processed words for every article

In [886]:
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [887]:
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [888]:
def labelled_sentences(articles, label_type):
    labelledSentences = []
    for i,_d in enumerate(articles):
        labelledSentences.append(LabeledSentence(_d, label_type[i]))
    return labelledSentences

In [889]:
df_train_labelled=labelled_sentences(df_train['statement'],df_train['label'])
df_train_labelled[1]

LabeledSentence(words=['declin', 'coal', 'start', 'start', 'natur', 'ga', 'took', 'start', 'begin', 'presid', 'georg', 'bush', 'administr'], tags='half-true')

In [890]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_train['statement'])]

In [891]:
tagged_data_words=[x.words for x in tagged_data]
tagged_data_words_1D=[]
for row in range(len(tagged_data_words)):
    for col in range(len(tagged_data_words[row])):
        tagged_data_words_1D.append(tagged_data_words[row][col]) 

In [892]:
n_dim=500
w2v = Word2Vec(size=n_dim, min_count=0)
w2v.build_vocab(tagged_data_words)
w2v.train(tagged_data_words,total_examples=model.corpus_count,epochs=model.epochs)

(476232, 524220)

In [893]:
w2v.most_similar('polit')

[('washington', 0.9999792575836182),
 ('think', 0.9999786615371704),
 ('report', 0.9999765157699585),
 ('militari', 0.9999760389328003),
 ('chang', 0.9999760389328003),
 ('mayor', 0.99997478723526),
 ('made', 0.9999739527702332),
 ('rule', 0.999973714351654),
 ('union', 0.9999733567237854),
 ('group', 0.9999725222587585)]

In [894]:
df_train_statements=df_train[['statement','label']]
df_train_statements_vectorized=df_train_statements['statement'].map(lambda x: [w2v[w] for w in x])

In [895]:
df_train_statements['statement']=df_train_statements_vectorized

In [896]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_train_statements['label']=le.fit_transform(df_train_statements['label'])

In [897]:
df_train_statements.head()

,statement,label
0,"[[-0.071273394, -0.21980503, -0.17088895, -0.3...",1
1,"[[-0.009013376, -0.036269505, -0.03342471, -0....",2
2,"[[-0.056503322, -0.15603647, -0.106545195, -0....",3
3,"[[-0.07002489, -0.13010412, -0.18100198, -0.32...",1
4,"[[-0.037856176, -0.13625227, -0.12308405, -0.2...",2


In [898]:
#list(le.inverse_transform(df_train_statements['label']))

In [899]:
def document_vector(doc):
    return np.mean(doc, axis=0)

In [900]:
df_train_statements['statement']=df_train_statements['statement'].apply(document_vector)
df_train_statements.drop(index=4497,inplace=True)
df_train_statements.reset_index(drop=True)

,statement,label
0,"[-0.033352505, -0.115471765, -0.1008389, -0.20...",1
1,"[-0.03209122, -0.11188347, -0.09611944, -0.200...",2
2,"[-0.0416491, -0.13409616, -0.11471135, -0.2376...",3
3,"[-0.04055827, -0.12158102, -0.12187575, -0.233...",1
4,"[-0.029154817, -0.1044416, -0.09467153, -0.185...",2
5,"[-0.024889307, -0.09717098, -0.09060201, -0.17...",5
6,"[-0.02782396, -0.10775762, -0.10003243, -0.190...",0
7,"[-0.0264311, -0.09619983, -0.08903993, -0.1716...",2
8,"[-0.023465768, -0.08304977, -0.07590949, -0.14...",2
9,"[-0.027807422, -0.09717227, -0.08934019, -0.17...",3


In [901]:
arr1=[]
for x in df_train_statements['statement'].values:
    arr1.append(x.tolist())   
X=pd.DataFrame(arr1)
y=df_train_statements[['label']]
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [902]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
3474,-0.027978,-0.091612,-0.083095,-0.170026,-0.061120,-0.129369,0.127160,-0.087610,-0.005433,-0.162188,...,0.028630,-0.231982,0.101287,-0.035126,0.056460,-0.101073,-0.062534,-0.196464,-0.005472,-0.147555
6202,-0.032392,-0.118972,-0.107899,-0.212723,-0.070173,-0.155286,0.164647,-0.110720,-0.012463,-0.200998,...,0.031655,-0.291024,0.119304,-0.046021,0.074330,-0.122201,-0.080250,-0.237642,-0.013952,-0.186100
5868,-0.023381,-0.079051,-0.067456,-0.140094,-0.051582,-0.107688,0.107850,-0.073287,-0.003735,-0.133876,...,0.024366,-0.187382,0.082109,-0.029376,0.049344,-0.083538,-0.053393,-0.161325,-0.002296,-0.120688
7357,-0.040725,-0.152912,-0.138666,-0.277544,-0.096480,-0.204488,0.210537,-0.142458,-0.013181,-0.264635,...,0.042773,-0.374641,0.158365,-0.063352,0.096612,-0.159552,-0.104370,-0.313410,-0.012780,-0.241565
9514,-0.031611,-0.107082,-0.096570,-0.193964,-0.069467,-0.144236,0.147663,-0.100687,-0.011222,-0.186302,...,0.031470,-0.263172,0.113781,-0.042725,0.067223,-0.113685,-0.071133,-0.222092,-0.008908,-0.168860


In [903]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
lr_pred = logisticRegr.predict(X_test)

In [904]:
from sklearn import metrics
# Lets print the confusion matrix
print(metrics.confusion_matrix(y_test,lr_pred))

[[  1 179 212  60   0   0]
 [  1 248 273  91   0   0]
 [  0 214 304 117   0   0]
 [  0 180 267 157   0   0]
 [  0 135  98  23   0   0]
 [  0 178 220 114   0   0]]


In [905]:
print(metrics.classification_report(y_test,lr_pred))

             precision    recall  f1-score   support

          0       0.50      0.00      0.00       452
          1       0.22      0.40      0.28       613
          2       0.22      0.48      0.30       635
          3       0.28      0.26      0.27       604
          4       0.00      0.00      0.00       256
          5       0.00      0.00      0.00       512

avg / total       0.22      0.23      0.17      3072



In [906]:
from sklearn.ensemble import RandomForestClassifier
# Lets try with 100 trees
num_trees=100
rf=RandomForestClassifier(n_estimators=num_trees)
rf.fit(X_train, y_train)
rf_pred=rf.predict(X_test)

In [907]:
print(metrics.confusion_matrix(y_test,lr_pred))

[[  1 179 212  60   0   0]
 [  1 248 273  91   0   0]
 [  0 214 304 117   0   0]
 [  0 180 267 157   0   0]
 [  0 135  98  23   0   0]
 [  0 178 220 114   0   0]]


In [908]:
print(metrics.classification_report(y_test,lr_pred))

             precision    recall  f1-score   support

          0       0.50      0.00      0.00       452
          1       0.22      0.40      0.28       613
          2       0.22      0.48      0.30       635
          3       0.28      0.26      0.27       604
          4       0.00      0.00      0.00       256
          5       0.00      0.00      0.00       512

avg / total       0.22      0.23      0.17      3072



## Doc2Vec

In [909]:
from gensim.models.doc2vec import Doc2Vec

In [910]:
df_train_statements_d2v=df_train[['statement','label']]

In [911]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("d2v.model")
print("Model Saved")    

Model Saved


In [916]:
model= Doc2Vec.load("d2v.model")
arr1=[]
for x in df_train_statements_d2v['statement']:
    arr1.append(model.infer_vector(x))

In [917]:
X=pd.DataFrame(arr1)
y=df_train_statements_d2v[['label']]
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [918]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
3474,-0.304127,0.825729,0.312745,-1.475605,-0.809256,-0.231703,0.448742,0.457160,-0.327754,-1.460855,-0.985821,0.442876,0.414575,0.383386,1.150307,0.837847,-0.688943,-0.187033,-0.118576,-0.255594
6202,0.039357,-0.418888,0.091556,0.354028,0.342485,-0.195747,-0.157043,-0.003259,0.266163,-0.484096,-0.419687,0.315179,-0.457909,-0.190802,1.219846,-0.580350,-0.110506,0.295929,0.264127,0.426879
5868,-0.619638,-0.206327,-0.960705,0.470735,-0.767726,-0.969793,-0.443377,-1.277578,0.352402,-1.028049,0.587345,-0.249423,0.192046,0.648746,1.272122,-0.112119,0.178527,0.124693,0.536602,1.150931
7357,-0.015409,-0.345323,-0.103448,-0.719424,0.466388,-0.097204,-0.485970,-0.618499,-0.540021,-0.366585,0.382142,-0.087365,-0.076497,-0.227300,0.476708,0.328608,-0.330903,0.409051,1.052046,0.335144
9514,0.147992,-0.176789,-0.651227,-0.242075,-0.224525,-0.513575,0.980304,-0.331123,-0.060687,-0.158046,0.021537,-0.421786,0.154784,-0.702727,0.732718,0.226165,-0.066381,0.266357,0.456416,0.122346


In [926]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(C=100)
logisticRegr.fit(X_train, y_train)
lr_pred = logisticRegr.predict(X_test)

In [927]:
print(metrics.confusion_matrix(y_test,lr_pred))

[[ 49 144 163  93   0  18]
 [ 57 218 184 128   0  25]
 [ 62 182 231 139   0  35]
 [ 62 169 189 137   0  29]
 [ 28  96  59  61   0   8]
 [ 33 155 165 125   0  28]]


In [928]:
print(metrics.classification_report(y_test,lr_pred))

             precision    recall  f1-score   support

barely-true       0.17      0.10      0.13       467
      false       0.23      0.36      0.28       612
  half-true       0.23      0.36      0.28       649
mostly-true       0.20      0.23      0.22       586
 pants-fire       0.00      0.00      0.00       252
       true       0.20      0.06      0.09       506

avg / total       0.19      0.22      0.19      3072



In [929]:
from sklearn.ensemble import RandomForestClassifier
# Lets try with 100 trees
num_trees=100
rf=RandomForestClassifier(n_estimators=num_trees)
rf.fit(X_train, y_train)
rf_pred=rf.predict(X_test)

In [930]:
print(metrics.confusion_matrix(y_test,lr_pred))

[[ 49 144 163  93   0  18]
 [ 57 218 184 128   0  25]
 [ 62 182 231 139   0  35]
 [ 62 169 189 137   0  29]
 [ 28  96  59  61   0   8]
 [ 33 155 165 125   0  28]]


In [925]:
print(metrics.classification_report(y_test,lr_pred))

             precision    recall  f1-score   support

barely-true       0.17      0.10      0.13       467
      false       0.23      0.36      0.28       612
  half-true       0.23      0.36      0.28       649
mostly-true       0.20      0.23      0.22       586
 pants-fire       0.00      0.00      0.00       252
       true       0.20      0.06      0.09       506

avg / total       0.19      0.22      0.19      3072



In [ ]:
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)